#### Training Pipeline

In [1]:
# Importing necessary libraries
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

import torch

In [2]:
# Loading the data
df = pd.read_csv('https://raw.githubusercontent.com/gscdit/Breast-Cancer-Detection/refs/heads/master/data.csv')
df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [3]:
# Splitting the data
df.drop(columns=['id', 'Unnamed: 32'], inplace= True)
X_train, X_test, y_train, y_test = train_test_split(df.iloc[:, 1:], df.iloc[:, 0], test_size=0.2)

In [4]:
# Preprocessing

# Scaling
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Label Encoding
encoder = LabelEncoder()
y_train = encoder.fit_transform(y_train)
y_test = encoder.transform(y_test)

In [6]:
# Device agnostic code
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device, torch.cuda.get_device_name())

cuda NVIDIA GeForce MX350


In [7]:
# From numpy arrays to pytorch tensors
X_train_tensor = torch.from_numpy(X_train)
X_test_tensor = torch.from_numpy(X_test)
y_train_tensor = torch.from_numpy(y_train)
y_test_tensor = torch.from_numpy(y_test)

In [8]:
X_train_tensor.shape

torch.Size([455, 30])

In [9]:
y_train_tensor.shape

torch.Size([455])

In [10]:
# Model Architecture
class MySimpleNN():
    def __init__(self, X):
        # Randomly initialize weights and biases
        self.weights = torch.rand(X.shape[1], 1, dtype=torch.float64, requires_grad=True)
        self.bias = torch.zeros(1, dtype=torch.float64, requires_grad=True)

    def forward(self, X):
        # Forward Proporgation and conversion to propabilities
        z = torch.matmul(X, self.weights) + self.bias
        y_pred = torch.sigmoid(z)
        return y_pred

    def loss_function(self, y_pred, y):
        # Clamp predictions to avoid log(0) and log(1)
        epsilon = 1e-8
        y_pred = torch.clamp(y_pred, epsilon, 1 - epsilon)

        # Calculate loss
        loss = -(y_train_tensor * torch.log(y_pred) + (1 - y_train_tensor) * torch.log(1 - y_pred)).mean()
        return loss

In [11]:
# Parameters for optimization
learning_rate = 0.1
epochs = 25

In [12]:
# create model
model = MySimpleNN(X_train_tensor)

# define loop
for epoch in range(epochs):
    # forward pass
    y_pred = model.forward(X_train_tensor)

    # loss calculate
    loss = model.loss_function(y_pred, y_train_tensor)

    # backward pass
    loss.backward()

    # parameters update
    with torch.no_grad():
        model.weights -= learning_rate * model.weights.grad
        model.bias -= learning_rate * model.bias.grad

    # zero gradients
    model.weights.grad.zero_()
    model.bias.grad.zero_()

    # print loss in each epoch
    print(f'Epoch: {epoch + 1}, Loss: {loss.item()}')

Epoch: 1, Loss: 3.5913515144314996
Epoch: 2, Loss: 3.432998722446844
Epoch: 3, Loss: 3.2709514120417706
Epoch: 4, Loss: 3.1072198994398077
Epoch: 5, Loss: 2.938432693623911
Epoch: 6, Loss: 2.764996537633055
Epoch: 7, Loss: 2.5834169599495658
Epoch: 8, Loss: 2.4025189145980983
Epoch: 9, Loss: 2.223509238923579
Epoch: 10, Loss: 2.049933057640848
Epoch: 11, Loss: 1.8798573623603585
Epoch: 12, Loss: 1.7190275462145541
Epoch: 13, Loss: 1.5690855739365233
Epoch: 14, Loss: 1.431776572769955
Epoch: 15, Loss: 1.3088019066010852
Epoch: 16, Loss: 1.2016059626738163
Epoch: 17, Loss: 1.1111021398529823
Epoch: 18, Loss: 1.0373493825645967
Epoch: 19, Loss: 0.9792722509431545
Epoch: 20, Loss: 0.9346947660993434
Epoch: 21, Loss: 0.900848484321334
Epoch: 22, Loss: 0.875020659290233
Epoch: 23, Loss: 0.8549451541449422
Epoch: 24, Loss: 0.838907122173219
Epoch: 25, Loss: 0.8256935050251046


In [13]:
model.bias

tensor([-0.1513], dtype=torch.float64, requires_grad=True)

In [14]:
model.weights

tensor([[-0.3254],
        [ 0.2149],
        [-0.3249],
        [-0.0101],
        [ 0.0663],
        [-0.3504],
        [-0.4797],
        [ 0.1276],
        [-0.0184],
        [-0.0886],
        [-0.1134],
        [-0.2043],
        [ 0.4547],
        [-0.0104],
        [ 0.5951],
        [ 0.1851],
        [ 0.0426],
        [-0.4376],
        [ 0.2119],
        [ 0.0801],
        [ 0.0823],
        [ 0.5311],
        [-0.2292],
        [ 0.5587],
        [ 0.2254],
        [ 0.3029],
        [-0.2449],
        [ 0.3819],
        [-0.2634],
        [ 0.3533]], dtype=torch.float64, requires_grad=True)

In [15]:
# model evaluation
with torch.no_grad():
    y_pred = model.forward(X_test_tensor)
    y_pred = (y_pred > 0.5).float()
    accuracy = (y_pred == y_test_tensor).float().mean()
    print(f'Accuracy: {accuracy.item()}')

Accuracy: 0.5492458939552307
